### Rag Pipelines- Data Ingestion to Vector Db pipelines

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

C:\Users\pranj\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

### Read all the pdf's inside the dictionary
def process_all_pdfs(pdf_directory):
    """Process all the pdf files in a dictionary"""
    all_documents = []
    pdf_dir = Path(pdf_directory)

    # Find all the PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))

    print(f"Found{len(pdf_files)} PDF files to process")

    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()

            # Add source info to metadataa
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'

            all_documents.extend(documents)
            print(f" Loaded{len(documents)} pages")
        
        except Exception as e:
            print(f"Erro:{e}")

    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")


Found3 PDF files to process

Processing: TCS1.pdf
 Loaded3 pages

Processing: TCS2.pdf
 Loaded4 pages

Processing: TCS3.pdf
 Loaded37 pages

Total documents loaded: 44


In [3]:
all_pdf_documents

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-09T01:52:04+05:30', 'author': 'Pranjal Singh', 'moddate': '2025-11-09T01:52:04+05:30', 'source': '..\\data\\TCS1.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': 'TCS1.pdf', 'file_type': 'pdf'}, page_content='Tata Consultancy Services, commonly known as TCS, stands as one of the most prominent global \ntechnology and consulting organizations originating from India. It began its journey in 1968 \nunder the Tata Group and steadily expanded into a worldwide leader in digital and IT services. \nFrom its early days of handling basic computing tasks for Indian enterprises, TCS has \ntransformed into a powerhouse that supports the digital operations of major institutions across \nthe world. Its growth has been deeply rooted in a long-term vision, disciplined execution, value-\ndriven leadership, and a commitment to excellence in technology and business transf

In [4]:
## Text Splitting get into chunks

def split_documents(documents,chunk_size=800,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n","\n"," ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")

    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [5]:
chunks = split_documents(all_pdf_documents)
chunks

Split 44 documents into 111 chunks

Example chunk:
Content: Tata Consultancy Services, commonly known as TCS, stands as one of the most prominent global 
technology and consulting organizations originating from India. It began its journey in 1968 
under the Ta...
Metadata: {'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-09T01:52:04+05:30', 'author': 'Pranjal Singh', 'moddate': '2025-11-09T01:52:04+05:30', 'source': '..\\data\\TCS1.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': 'TCS1.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-11-09T01:52:04+05:30', 'author': 'Pranjal Singh', 'moddate': '2025-11-09T01:52:04+05:30', 'source': '..\\data\\TCS1.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1', 'source_file': 'TCS1.pdf', 'file_type': 'pdf'}, page_content='Tata Consultancy Services, commonly known as TCS, stands as one of the most prominent global \ntechnology and consulting organizations originating from India. It began its journey in 1968 \nunder the Tata Group and steadily expanded into a worldwide leader in digital and IT services. \nFrom its early days of handling basic computing tasks for Indian enterprises, TCS has \ntransformed into a powerhouse that supports the digital operations of major institutions across \nthe world. Its growth has been deeply rooted in a long-term vision, disciplined execution, value-\ndriven leadership, and a commitment to excellence in technology and business transf

In [6]:
## Embedding and VectorStore DB
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""


    def __init__(self, model_name:str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
        model_name: Huggingface model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the sentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model Loaded successfully. Embedding dimension:{self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model{self.model_name}: {e}")
            raise

    def generate_embeddings(self,texts: List[str])-> np.ndarray:
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generate embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts,show_progress_bar=True)
        print(f"Generated embeddings with shape:{embeddings.shape}")
        return embeddings
    
    def get_embedding_dimension(self)->int:
        """Get the embeding dimension of the model"""
        if not self.model:
            raise ValueError("Model not loaded")
        return self.model.get_sentence_embedding_dimension()
    

## Intialize the embedding Manager
embedding_manager=EmbeddingManager()
embedding_manager


Loading embedding model: all-MiniLM-L6-v2
Model Loaded successfully. Embedding dimension:384


### Vector Store

In [8]:
class VectorStore:
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store

        Args:
        collection_name: name of the ChromaDb collection
        persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            os.makedirs(self.persist_directory, exist_ok=True)

            self.client = chromadb.PersistentClient(path=self.persist_directory)

            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"},
                embedding_function=None     # We'll supply our own embeddings
            )

            print(f" Vector store initialized. Collection: {self.collection_name}")
            print(f" Total stored docs: {self.collection.count()}")

        except Exception as e:
            print(f" Error initializing vector store: {e}")
            raise
    
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        if len(documents) != len(embeddings):
            raise ValueError("No. of documents must match number of embeddings")

        print(f"Adding {len(documents)} documents to vetor store...")

        # Prepare data for ChromaDb
        ids = []
        metadata_list =[]
        documents_text = []
        embeddings_list = []

        for i,(doc,embedding) in enumerate(zip(documents,embeddings)):
            #Generate new id
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # Preparing metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadata_list.append(metadata)

            #Document content
            documents_text.append(doc.page_content)

            # Enbedding
            embeddings_list.append(embedding.tolist())

            # Add to collection
            try:
                self.collection.add(
                    ids=ids,
                    embeddings=embeddings_list,
                    metadatas=metadata_list,
                    documents=documents_text
                )
                print(f"Successfully add {len(documents)} documents to vector store")
                print(f"Total documents in collection:{self.collection.count()}")
            except Exception as e:
                print(f"Error adding documents to vector store:{e}")
                raise

vectorestore = VectorStore()
vectorestore


 Vector store initialized. Collection: pdf_documents
 Total stored docs: 222


In [10]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

### Generate the embeddings
embeddings = embedding_manager.generate_embeddings(texts)

### Store in vector Database
vectorestore.add_documents(chunks,embeddings)


Generate embeddings for 111 texts...


Batches: 100%|██████████| 4/4 [00:04<00:00,  1.08s/it]


Generated embeddings with shape:(111, 384)
Adding 111 documents to vetor store...
Successfully add 111 documents to vector store
Total documents in collection:223
Successfully add 111 documents to vector store
Total documents in collection:224
Successfully add 111 documents to vector store
Total documents in collection:225
Successfully add 111 documents to vector store
Total documents in collection:226
Successfully add 111 documents to vector store
Total documents in collection:227
Successfully add 111 documents to vector store
Total documents in collection:228
Successfully add 111 documents to vector store
Total documents in collection:229
Successfully add 111 documents to vector store
Total documents in collection:230
Successfully add 111 documents to vector store
Total documents in collection:231
Successfully add 111 documents to vector store
Total documents in collection:232
Successfully add 111 documents to vector store
Total documents in collection:233
Successfully add 111 docume

### Retrievel pipeline from vector store


In [22]:
class RAGRetriever:
    def __init__(self, embedding_manager: EmbeddingManager, vector_store=vectorestore):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query:str, top_k=5,score_threshold:float = 0.0)-> List[Dict[str,Any]]:
        """
        Retrieve relevant documents for a query

        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold

        Return:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")

        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        # Search in vector Store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )

            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i,(doc_id,document,metadata,distance) in enumerate(zip(ids,documents,metadatas,distances)):
                    # Convert distance to similarity score (ChromaDb uses Cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id':doc_id,
                            'content':document,
                            'metadata':metadata,
                            'similarity_score':similarity_score,
                            'distance': distance,
                            'rank':i+1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")

            return retrieved_docs
        
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return[]

rag_retriever = RAGRetriever(embedding_manager,vectorestore)
        


In [23]:
rag_retriever

In [29]:
rag_retriever.retrieve("Who is the founder of Tata?")

Retrieving documents for query: 'Who is the founder of Tata?'
Top K: 5, Score threshold: 0.0
Generate embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.40it/s]

Generated embeddings with shape:(1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_35494c4b_14',
  'content': 'The Tata Group, founded in 1868 by Jamsetji Tata, is one of India’s oldest and most respected \nbusiness conglomerates, known for its values, long-term vision, and deep contribution to India’s \neconomic and social development. Over more than a century, the gr oup has expanded into \nmultiple industries and built trusted companies that operate across the world. Tata Sons serves \nas the principal holding company and promoter of major Tata enterprises, guiding strategic \ndecisions and maintaining the group’s legacy. Tata companies operate independently under their \nown boards and leadership teams, but all follow the shared Tata values of integrity, customer \ntrust, quality, and nation -building. Many of these companies are global leaders in their sectors',
  'metadata': {'source_file': 'TCS2.pdf',
   'page': 0,
   'file_type': 'pdf',
   'content_length': 759,
   'page_label': '1',
   'total_pages': 4,
   'creator': 'Microsoft® Word 2021',
   '

### Integration VectorDB contect pipeline with LLM output

In [ ]:
# 2. Simple RAG function: retrieve function + generate response
def rag_simple(query,retriever,llm,top_k=3):
    results = retriever.retrieve(query,top_k=top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question"
    
    # Generate the answer using GROQ LLM
    prompt =f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    response = llm.invoke([prompt.format(context,query=query)])
    return response.content